In [1]:
from ggplot import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

%matplotlib inline

In [2]:
df_2015 = pd.read_csv('./data/2015_WH_pay_links_merged_edited.csv')
df_2015.loc[df_2015.Gender=='?','Gender'] = np.nan
df_2015 = df_2015[ pd.notnull(df_2015['Gender']) ]


df_long = pd.read_csv('./data/combined_wh_pay.csv')
df_long['Name'] = df_long['Name'].str.strip()
df_long = df_long[ df_long.Status != "Employee (part-time)"]

df_long.loc[ df_long.Gender =='?', 'Gender'] = np.nan
df_long.loc[ df_long.Salary ==0, 'Gender'] = np.nan
df_long = df_long.dropna()

sal_cutoff = 70000

In [3]:
### Histograms for analyst, staff assistant


In [4]:
newdf = df_long.drop_duplicates(subset=['Title','Cleaned_Name'])
a = newdf.Cleaned_Title.value_counts()
b = a.index.tolist()

AttributeError: 'DataFrame' object has no attribute 'Cleaned_Title'

In [5]:
import time
from itertools import islice

idx = 9

title = a.index[idx]
print 
print title, a[idx]
ggplot(newdf[newdf.Cleaned_Title==title], aes('Salary', fill='Gender'))\
+ geom_histogram(binwidth=5000)\
+ scale_x_continuous(limits = (0,250000))

NameError: name 'a' is not defined

In [6]:
#df[df['A'].isin([3, 6])]

low_var_jobs = ['ANALYST', 'STAFF ASSISTANT']
#newdf = df_long[ df_long['Title'].isin(low_var_jobs)]
newdf = df_long[ df_long['Title']==low_var_jobs[0]]

p = ggplot(newdf, aes('Salary', color='Gender',fill='Gender')) +\
geom_histogram(binwidth=5000) +\
scale_x_continuous(limits = (0, 250000)) +\
scale_colour_manual(values=color_list) +\
ggtitle('Analyst')
#ylab("Frequency")
#facet_wrap('Title') +\

ggsave(p, "./figs/analyst.png")

NameError: name 'color_list' is not defined

In [7]:
#df[df['A'].isin([3, 6])]

low_var_jobs = ['ANALYST', 'STAFF ASSISTANT']
#newdf = df_long[ df_long['Title'].isin(low_var_jobs)]
newdf = df_long[ df_long['Title']==low_var_jobs[1]]

p = ggplot(newdf, aes('Salary', color='Gender',fill='Gender')) +\
geom_histogram(binwidth=5000)\
+ scale_x_continuous(limits = (0, 250000))\
+ scale_colour_manual(values=color_list)\
+ ggtitle('Staff Assistant')
#ylab("Frequency")
#facet_wrap('Title') +\

ggsave(p, "./figs/staff_assistant.png")

NameError: name 'color_list' is not defined

In [8]:
### Salary histogram for senior policy advisor


In [9]:
jobs = ['SENIOR POLICY ADVISOR']
newdf = df_long[ df_long['Title'].isin(jobs)]

p = ggplot(newdf, aes('Salary', fill='Gender', color='Gender'))\
+ geom_histogram(alpha=0.9)\
+ scale_x_continuous(limits = (0, 250000))\
+ ylab("Frequency")\
+ ggtitle("Senior Policy Advisor")\
+ scale_colour_manual(values=color_list)

ggsave(p, "./figs/senior_policy_advisor.png")

NameError: name 'color_list' is not defined

In [10]:
### Logistic Regression


In [11]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

def check_alphnum(char):
    char = ord(char)
    if (char >= 48) & (char <= 57):
        return True
    elif (char >= 65) & (char <= 90):
        return True
    elif (char >= 97) & (char <= 122):
        return True
    elif char == 32:
        return True
    else:
        return False
    
def check_word(word):
    return "".join([char for char in word if check_alphnum(char)])

newdf = df_long.drop_duplicates(subset=['Title','Cleaned_Name'])

In [12]:
stop = stopwords.words('english')
#df = df[ df['Salary']<80000]
titles = newdf['Title'].str.split().apply(lambda x: [item.lower() for item in x])
titles = titles.apply(lambda sentence: [check_word(word) for word in sentence])
newdf['Cleaned_Title'] = titles.apply(lambda x: " ".join(x))
vectorizer = CountVectorizer(analyzer = 'word',
                            ngram_range = (1,10),
                            max_features = 5000)
Xts = vectorizer.fit_transform( newdf['Cleaned_Title'] )


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
from sklearn.linear_model import LogisticRegression
from scipy import sparse
from sklearn import metrics, cross_validation

sex = newdf['Gender'].apply(lambda x: 0 if x=='male' else 1).astype(int)
model = LogisticRegression()

SEED = 4
N = 10
result_list = []
for i in range(N):
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(Xts, sex, test_size=0.2,\
        random_state = i*SEED)
    model.fit(X_train, y_train)
#    preds_proba = model.predict_proba(Xts)
    preds = model.predict(X_cv)    
    result = [(x ^ y) for x, y in zip(preds, y_cv)]
    result_list.append(1-sum(result)/float(len(result)))

print "vanilla logistic regression", sum(result_list)/float(len(result_list))
print "base case", newdf.Gender.value_counts()[0] / float(len(newdf))


#print metrics.f1_score(sex, preds)

vanilla logistic regression 0.580825242718
base case 0.514591439689


In [14]:
a = sorted(zip(vectorizer.get_feature_names(), model.coef_[0]), \
           key=lambda x:x[1], reverse=True)